In [2]:
import numpy as np
import pandas as pd
import vectorbtpro as vbt

from vctr.data.data_loader import get_data
from vctr.data.labeling import label_data
from vctr.features.feature_engineering import add_features

In [3]:
df = get_data('FTM', '1h')


In [4]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objs as go
from plotly.subplots import make_subplots


def calc_ema(series, period):
    series = pd.Series(series)
    ema = ta.ema(series, period)
    # Calculate the expanding mean (simple average) up to each point in the 'close' series
    expanding_mean = series.expanding().mean()
    # Replace the NaN values in the SMA with the corresponding expanding mean values
    ema = ema.combine_first(expanding_mean)
    ema.index = df.index
    return ema


# RSI settings
rsi_length = 14
rsi_source = df['close']

# MA settings
ma_type = 'SMA'  # You can change this to any of the supported types: "SMA", "Bollinger Bands", "EMA", "SMMA (RMA)", "WMA", "VWMA"
ma_length = 14
bb_stddev = 2.0

# Calculate RSI
delta = rsi_source.diff()
gain = np.where(delta > 0, delta, 0)
loss = np.where(delta < 0, -delta, 0)

avg_gain = calc_ema(gain, rsi_length)
avg_loss = calc_ema(loss, rsi_length)
rs = avg_gain / avg_loss

rsi = 100 - (100 / (1 + rs))

# Calculate moving average
if ma_type == 'SMA':
    rsi_ma = ta.sma(rsi, ma_length)
elif ma_type == 'Bollinger Bands':
    rsi_ma = ta.sma(rsi, ma_length)
elif ma_type == 'EMA':
    rsi_ma = ta.ema(rsi, ma_length)
elif ma_type == 'SMMA (RMA)':
    rsi_ma = ta.rma(rsi, ma_length)
elif ma_type == 'WMA':
    rsi_ma = ta.wma(rsi, ma_length)
elif ma_type == 'VWMA':
    rsi_ma = ta.vwma(df['close'], df['volume'], ma_length)

# Bollinger Bands
if ma_type == 'Bollinger Bands':
    rsi_std = ta.stdev(rsi, ma_length)
    bb_upper_band = rsi_ma + rsi_std * bb_stddev
    bb_lower_band = rsi_ma - rsi_std * bb_stddev
